In [164]:
import pandas as pd

In [165]:
df = pd.read_csv('dataset_test_ML.csv')

In [166]:
df.columns

Index(['id', 'title', 'genres', 'cast', 'release_year', 'first_actors',
       'top_actors_str'],
      dtype='object')

In [167]:
df[df.duplicated(subset='title',keep=False)]

,id,title,genres,cast,release_year,first_actors,top_actors_str


In [104]:
df.drop_duplicates(subset='title',keep='first',inplace=True)

In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42192 entries, 0 to 42191
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              42192 non-null  int64 
 1   title           42192 non-null  object
 2   genres          42192 non-null  object
 3   cast            42192 non-null  object
 4   release_year    42192 non-null  int64 
 5   first_actors    42192 non-null  object
 6   top_actors_str  39954 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.3+ MB


In [60]:
#df.to_csv('dataset_test_ML.csv',index=False)

In [69]:
#df['genres']=pd.Categorical(df['genres']).codes

In [71]:
#df['genres']=df.genres.astype(str)

In [155]:
import ast

# Definir una función para extraer los primeros tres actores de cada registro
def extract_actors(cast):
    cast_list = ast.literal_eval(cast)  # Convertir la cadena en una lista
    return cast_list[:3]  # Devolver los primeros tres elementos de la lista

# Aplicar la función a la columna 'cast' y crear una nueva columna 'first_actors'
df['first_actors'] = df['cast'].apply(extract_actors)


In [169]:
type(df.first_actors[0])

str

In [157]:
#df['top_actors_str']=df['first_actors'].astype(str)

In [170]:
df.top_actors_str[0]

'Tom Hanks, Tim Allen, Don Rickles'

In [159]:
#df['top_actors_str'] = df['top_actors_str'].str.replace('[', '').str.replace(']', '')
#df['top_actors_str'] = df['top_actors_str'].str.replace("'", "").str.replace("'", "")
#df['top_actors_str']=df['top_actors_str'].fillna('ND')

In [163]:
#df.to_csv('dataset_test_ML.csv',index=False)

In [194]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

def recomendacion_4(titulo):
            
    # Obtener el índice de la película que coincide con el título ingresado (case insensitive)
    indices = pd.Series(df.index, index=df['title'].str.lower()).drop_duplicates()
    titulo_lower = titulo.lower()
    
    if titulo_lower not in indices:
        return "Título no encontrado"
    
    idx = indices[titulo_lower]
    
    # Seleccionar las características relevantes para el algoritmo KNN
    features = ['genres']
    X = df[features]
    
    # Convertir las listas en columnas binarias utilizando MultiLabelBinarizer
    mlb = MultiLabelBinarizer()
    encoded_genres = pd.DataFrame(mlb.fit_transform(X['genres']), columns=mlb.classes_, index=X.index)
    
    # Convertir los nombres de características numéricas a cadenas
    numeric_features = X.drop('genres', axis=1)
    numeric_features.columns = numeric_features.columns.astype(str)
    
    # Aplicar PCA para reducir la dimensionalidad
    #El PCA es una técnica de reducción de dimensionalidad que 
    # transforma un conjunto de variables correlacionadas en un conjunto más pequeño de variables no correlacionadas 
    # llamadas componentes principales.
    pca = PCA(n_components=0.6)  # Mantenemos el 60% de la varianza explicada
    encoded_genres_pca = pca.fit_transform(encoded_genres)
    
    # Concatenar las características numéricas y las características transformadas por PCA
    X = pd.concat([numeric_features, pd.DataFrame(encoded_genres_pca)], axis=1)
    
    # Normalizar las características
    X = (X - X.mean()) / X.std()
    
    # Crear una instancia del algoritmo KNN
    knn = NearestNeighbors(n_neighbors=6)  # Consideramos 6 vecinos, incluyendo la película seleccionada
    
    # Resolvemos el siguiente error con la recomendacion de la propia libreria:  
    # Feature names are only supported if all input features have string names, but your input has ['int', 'str'] 
    # as feature name / column name types. If you want feature names to be stored and validated, 
    # you must convert them all to strings, by using: =>>>>> X.columns = X.columns.astype(str)
    #X.columns = X.columns.astype(str)
    
    # Entrenar el modelo KNN
    knn.fit(X)
    
    # Obtener la distancia y los índices de los vecinos más cercanos
    distances, indices = knn.kneighbors(X.iloc[idx].values.reshape(1, -1))
    
    # Obtener los títulos de las películas más similares (excluyendo la película seleccionada)
    similar_movies = df['title'].iloc[indices[0][1:6]]
    
    return similar_movies

In [199]:
recomendacion_4('Happy feet')

15278                                       Big Buck Bunny
1349                       Beavis and Butt-Head Do America
10052    Family Guy Presents Stewie Griffin: The Untold...
12920                 Futurama: Into the Wild Green Yonder
21841                                     The Cat Concerto
Name: title, dtype: object